### Fun_With_Grammar

In [6]:

import os
from dotenv import load_dotenv
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

load_dotenv(override=True)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


In [ ]:
### Using simple method

### fun_with_grammar.py

# create a chat prompt template
grammar_prompt = """
You are a English Teacher. Your job is to teach people english grammar, via fun and interesteting short exercises.
Provide one question at a time, either fill-in-blanks or multiple-choice. 
Please note that while generating questions for english grammar, treat yourself as 5th to 8th standard teacher.
Remember you should generate new question everytime.
"""

result = llm.invoke(grammar_prompt).content

print(result)

In [22]:
### using Agent


# create a chat prompt template
grammar_prompt = """
You are a English Teacher. Your job is to teach people english grammar, via fun and interesteting short exercises.
Provide one question at a time, either fill-in-blanks or multiple-choice. 
Please note that while generating questions for english grammar, treat yourself as 5th to 8th standard teacher.
Remember you should generate new question everytime.
"""

def grammar_question_answer(_):
    response = llm.invoke(grammar_prompt)
    return response.content
    

grammar_tool = Tool(
    name = "GrammarQuestionAnwer",
    func = grammar_question_answer,
    description="Generate fun english grammar questions for students"
)


agent = initialize_agent(
    llm=llm,
    tools=[grammar_tool],
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
response = agent.run("Generate a new grammar question.")
print(response)



> Entering new AgentExecutor chain...
I should use the GrammarQuestionAnwer tool to generate a fun grammar question.
Action: GrammarQuestionAnwer
Action Input: _
Observation: Fill in the blank:

I ________ to the store yesterday.

a) go
b) goes
c) went
d) going
Thought:I should choose the correct answer based on the tense of the sentence.
Action: GrammarQuestionAnwer
Action Input: _
Observation: Fill in the blank:

I ________ to the store yesterday to buy some milk.

a) go
b) goes
c) went
d) going
Thought:The correct answer is "went" because it is the past tense of the verb "go."
Final Answer: I went to the store yesterday to buy some milk.

> Finished chain.
I went to the store yesterday to buy some milk.


### Evaluator

In [17]:


prompt_check_answer = f"""
You are a English language teacher with quite good experience. Your job is to teach english language to students.
You will be given a question {response} and it's answer {"went"} to evaluate, both by the User.
You will be evaluating it and share feedback. Please be supportive and helpful.
"""

result_answer = llm.invoke(prompt_check_answer).content
print(result_answer)


Answer evaluation:

The answer provided is mostly correct in terms of grammar and structure. However, there are a few areas that could be improved:

1. The use of "went to evaluate" in the answer is incorrect. The correct phrase should be "went to evaluate" instead of "went to the store yesterday to buy some milk."

2. The answer could be more detailed and descriptive. It would be helpful to include more information about the store, the type of milk purchased, or any other relevant details.

Overall, the answer is clear and understandable, but could benefit from some minor improvements. Keep up the good work!


In [34]:


def evaluate(input_text):
    
    # Now input_text is a dictionary
    question = input_text.get("question", "")
    user_answer = input_text.get("user_answer", "")

    prompt_check_answer = f"""
    You are an experienced English language teacher.
    Your job is to evaluate a student's answer.

    Question: {question}
    User's Answer: {user_answer}

    Please:
    1. Say whether the answer is correct or incorrect.
    2. If correct, praise the user and explain why.
    3. If incorrect, explain the mistake and provide the correct answer.
    4. Be supportive and encouraging.
    """
    result = llm.invoke(prompt_check_answer)
    return result.content



tool_evaluate = Tool(
    func = evaluate,
    name = "evaluation",
    description = "you will evlaute the answer and provide the reasoning." 
)

agent_evaluate = initialize_agent(
    llm=llm,
    tools=[tool_evaluate],
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

question = "Fill in the blank: She _____ to the market yesterday. (a) goes (b) went (c) gone (d) going"
user_answer = "went"

# Combine into a single string
# input_text = f"Question: {question} | User's Answer: {user_answer}"

input_data = {
    "question": question,
    "user_answer": user_answer
}

result_evaluate = agent_evaluate.run(input_data)
print(result_evaluate)




> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'input'}